In [1]:
ls

 Applying_a_pretrained_Sentiment_model_to_the_data.ipynb
 BiDirectionalLSTM_Sentiment.ipynb
 CFPB_Final.csv
 CFPB_Gradient.ipynb
 clean_full.zip
 complaints-2019-09-13_09_37.7z
 complaints-2019-09-13_09_37.csv
 Consumer_Complaints.csv
 Copy_of_Mortgage_Data_Shared_Notebook.ipynb
 Copy_of_original_for_visualisations.ipynb
 demo_results.csv
'git prod'/
 LDA.ipynb
 LDA-Mortgage.ipynb
 LDA_Vikas/
 model/
'Sam'\''s_copy_(Base_Notebook)_of_Multi_Class_Text_Classification_LSTM_Consumer_complaints.ipynb'
 tokenizer.pickle
 training_2.csv
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled.ipynb
 UST_SmartOps_CFPB.pptx


In [2]:
#sentence,label

In [3]:
import pandas as pd
df = pd.read_csv('training_2.csv')

In [4]:
df.describe()

,sentence,label
count,49579,49579
unique,49579,2
top,Whether you're a fan of the series which inspi...,positive
freq,1,24883


In [5]:
df.head()

,sentence,label
0,A wonderful little production. The filming te...,positive
1,I thought this was a wonderful way to spend ti...,positive
2,Basically there's a family where a little boy ...,negative
3,"Petter Mattei's ""Love in the Time of Money"" is...",positive
4,"Probably my all-time favorite movie, a story o...",positive


In [6]:
import re

pat_1 = r"(?:\@|https?\://)\S+"
pat_2 = r'#\w+ ?'
combined_pat = r'|'.join((pat_1, pat_2))
www_pat = r'www.[^ ]+'
html_tag = r'<[^>]+>'
negations_ = {"isn't":"is not", "can't":"can not","couldn't":"could not", "hasn't":"has not",
                "hadn't":"had not","won't":"will not",
                "wouldn't":"would not","aren't":"are not",
                "haven't":"have not", "doesn't":"does not","didn't":"did not",
                 "don't":"do not","shouldn't":"should not","wasn't":"was not", "weren't":"were not",
                "mightn't":"might not",
                "mustn't":"must not"}
negation_pattern = re.compile(r'\b(' + '|'.join(negations_.keys()) + r')\b')

In [7]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [8]:
def data_cleaner(text):
    try:
        stripped = re.sub(combined_pat, '', text)
        stripped = re.sub(www_pat, '', stripped)
        cleantags = re.sub(html_tag, '', stripped)
        lower_case = cleantags.lower()
        neg_handled = negation_pattern.sub(lambda x: negations_[x.group()], lower_case)
        letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
        tokens = tokenizer.tokenize(letters_only)
        return (" ".join(tokens)).strip()
    except:
        return 'NC'

In [9]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

def post_process(data, n=1000000):
    data = data.head(n)
    data['sentence'] = data['sentence'].progress_map(data_cleaner)  
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [10]:
train = post_process(df)

progress-bar: 100%|██████████| 49579/49579 [00:07<00:00, 6213.97it/s]


In [11]:
train.columns

Index(['sentence', 'label'], dtype='object')

In [12]:
train

,sentence,label
0,a wonderful little production the filming tech...,positive
1,i thought this was a wonderful way to spend ti...,positive
2,basically there s a family where a little boy ...,negative
3,petter mattei s love in the time of money is a...,positive
4,probably my all time favorite movie a story of...,positive
5,i sure would like to see a resurrection of a u...,positive
6,this show was an amazing fresh andinnovative i...,negative
7,encouraged by the positive comments about this...,negative
8,if you like original gut wrenching laughter yo...,positive
9,phil the alien is one of those quirky films wh...,negative


In [13]:
train.loc[train['label'] == 'positive', 'train'] = 1

In [14]:
train['label'] = [1 if x =='positive' else 0 for x in train['label']] 

In [15]:
train.columns

Index(['sentence', 'label', 'train'], dtype='object')

In [16]:
ls

 Applying_a_pretrained_Sentiment_model_to_the_data.ipynb
 BiDirectionalLSTM_Sentiment.ipynb
 CFPB_Final.csv
 CFPB_Gradient.ipynb
 clean_full.zip
 complaints-2019-09-13_09_37.7z
 complaints-2019-09-13_09_37.csv
 Consumer_Complaints.csv
 Copy_of_Mortgage_Data_Shared_Notebook.ipynb
 Copy_of_original_for_visualisations.ipynb
 demo_results.csv
'git prod'/
 LDA.ipynb
 LDA-Mortgage.ipynb
 LDA_Vikas/
 model/
'Sam'\''s_copy_(Base_Notebook)_of_Multi_Class_Text_Classification_LSTM_Consumer_complaints.ipynb'
 tokenizer.pickle
 training_2.csv
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled.ipynb
 UST_SmartOps_CFPB.pptx


In [17]:
train.sentence[0]

'a wonderful little production the filming technique is very unassuming very old time bbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great master s of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell s murals decorating every surface are terribly well done'

In [18]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

train['Processed_Reviews'] = train.sentence.apply(lambda x: clean_text(x))

In [19]:
train

,sentence,label,train,Processed_Reviews
0,a wonderful little production the filming tech...,1,1.0,wonderful little production film technique una...
1,i thought this was a wonderful way to spend ti...,1,1.0,think wa wonderful way spend time hot summer w...
2,basically there s a family where a little boy ...,0,NaN,basically family little boy jake think zombie ...
3,petter mattei s love in the time of money is a...,1,1.0,petter mattei love time money visually stun fi...
4,probably my all time favorite movie a story of...,1,1.0,probably time favorite movie story selflessnes...
5,i sure would like to see a resurrection of a u...,1,1.0,sure would like see resurrection date seahunt ...
6,this show was an amazing fresh andinnovative i...,0,NaN,show wa amaze fresh andinnovative idea first a...
7,encouraged by the positive comments about this...,0,NaN,encourage positive comment film wa look forwar...
8,if you like original gut wrenching laughter yo...,1,1.0,like original gut wrench laughter like movie y...
9,phil the alien is one of those quirky films wh...,0,NaN,phil alien one quirky film humour base around ...


In [20]:
train.Processed_Reviews[0]

'wonderful little production film technique unassuming old time bbc fashion give comfort sometimes discomforting sense realism entire piece actor extremely well choose michael sheen ha get polari ha voice pat truly see seamless edit guide reference williams diary entry well worth watch terrificly write perform piece masterful production one great master comedy life realism really come home little thing fantasy guard rather use traditional dream technique remain solid disappear play knowledge sens particularly scene concern orton halliwell set particularly flat halliwell mural decorate every surface terribly well'

In [21]:
train.sentence[0]

'a wonderful little production the filming technique is very unassuming very old time bbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great master s of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell s murals decorating every surface are terribly well done'

In [22]:
df1 = train[['Processed_Reviews', 'label']]

In [23]:
df2 = train[['sentence', 'label']]#sentence

In [24]:
from sklearn.model_selection import train_test_split
x1Train, x1Test, y1Train, y1Test = train_test_split(df1.Processed_Reviews, df1.label, test_size = 0.2, random_state = 0)
x2Train, x2Test, y2Train, y2Test = train_test_split(df2.sentence, df2.label, test_size = 0.2, random_state = 0)

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x1Train)
list_tokenized_train = tokenizer.texts_to_sequences(x1Train)

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = y1Train

embed_size = 128
model1 = Sequential()
model1.add(Embedding(max_features, embed_size))
model1.add(Bidirectional(LSTM(32, return_sequences = True)))
model1.add(GlobalMaxPool1D())
model1.add(Dense(20, activation="relu"))
model1.add(Dropout(0.05))
model1.add(Dense(1, activation="sigmoid"))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model1.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Using TensorFlow backend.
/home/tom/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tom/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tom/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tom/.local/lib/python3.6/site-packages/tensorflow/python/framework/d

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 31730 samples, validate on 7933 samples
Epoch 1/3
31730/31730 [==============================] - 37s 1ms/step - loss: 0.3892 - acc: 0.8216 - val_loss: 0.2976 - val_acc: 0.8734
Epoch 2/3
31730/31730 [==============================] - 34s 1ms/step - loss: 0.2371 - acc: 0.9073 - val_loss: 0.2937 - val_acc: 0.8805
Epoch 3/3
31730/31730 [==============================] - 34s 1ms/step - loss: 0.1872 - acc: 0.9287 - val_loss: 0.3414 - val_acc: 0.8658


In [26]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x2Train)
list_tokenized_train = tokenizer.texts_to_sequences(x2Train)

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = y2Train

embed_size = 128
model2 = Sequential()
model2.add(Embedding(max_features, embed_size))
model2.add(Bidirectional(LSTM(32, return_sequences = True)))
model2.add(GlobalMaxPool1D())
model2.add(Dense(20, activation="relu"))
model2.add(Dropout(0.05))
model2.add(Dense(1, activation="sigmoid"))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model2.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 31730 samples, validate on 7933 samples
Epoch 1/3
31730/31730 [==============================] - 38s 1ms/step - loss: 0.4390 - acc: 0.7877 - val_loss: 0.3232 - val_acc: 0.8629
Epoch 2/3
31730/31730 [==============================] - 35s 1ms/step - loss: 0.2734 - acc: 0.8886 - val_loss: 0.3121 - val_acc: 0.8668
Epoch 3/3
31730/31730 [==============================] - 35s 1ms/step - loss: 0.2180 - acc: 0.9142 - val_loss: 0.3225 - val_acc: 0.8631


In [27]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

from keras.models import load_model

model2.save('model.h5')

In [28]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 810,537
Trainable params: 810,537
Non-trainable params: 0
_________________________________________________________________


In [29]:
for layer in model2.layers:
    print(layer)

In [30]:
y_test = y2Test
list_sentences_test = x2Test
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model2.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

F1-score: 0.8751807228915662
Confusion matrix:


array([[4081,  448],
       [ 847, 4540]])

In [31]:
y_pred

array([[False],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [False]])

In [32]:
train.label.value_counts()

1    24883
0    24696
Name: label, dtype: int64

In [33]:
y_test

166      0
44810    1
28233    0
1872     0
13503    1
11633    0
3769     0
36577    1
16235    0
39854    1
17827    1
43016    0
10322    1
19932    0
48442    0
2796     1
23819    0
23122    0
8348     1
44928    0
24474    1
18938    1
4064     1
9393     0
30627    0
1561     1
28884    1
47244    1
15473    1
33419    0
        ..
47022    0
7486     1
10826    0
43793    1
12590    0
37403    1
10085    0
24524    0
40779    0
36077    1
7308     1
34914    0
21791    1
4003     0
26136    0
5572     0
40536    1
43245    1
28530    1
30429    1
49031    0
5582     0
30299    0
35596    0
2862     0
1204     1
19223    1
28472    1
1559     1
21358    0
Name: label, Length: 9916, dtype: int64

In [34]:
prediction

array([[0.05618244],
       [0.9949334 ],
       [0.6289472 ],
       ...,
       [0.9667199 ],
       [0.9912505 ],
       [0.00553936]], dtype=float32)

In [35]:
print('Accuracy:')
accuracy_score(y_pred, y_test)
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(y_test, y_pred)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

Accuracy:
precision: [0.90108192 0.84276963]
recall: [0.828125   0.91018444]
fscore: [0.8630644  0.87518072]
support: [4928 4988]


In [36]:
accuracy_score(y_pred, y_test)

0.8694029850746269

In [37]:
to_predict = ["Although fairly involving as far as it goes , the film does n't end up having much that is fresh to say about growing up Catholic or , really , anything ",
              "What saves this deeply affecting film from being merely a collection of wrenching cases is Corcuera 's attention to detail",
              "The modern-day royals have nothing on these guys when it comes to scandals",
               "But believe it or not , it 's one of the most beautiful , evocative works I 've seen",
               "It was very good",
               "It was really good", 
                "It was really bad,", 
                "It is ok to keep them",
                "I was working with AmeriSave on a refinance, and after reading your report of they are doing the same thing. They intentionally mislead me on every step of the process -low interest rate, closing fees, which changed during the process. I had to pay up front just to move forward with the process, they assured me that this would be a quick, and that I had plenty of equity in my house, and that I had nothing to worry about. I started the process around and went back and forth with them until the loan was about to close on. The last phone call I had with them on the  the lender agreed with me and said they missed looking at my initial mortgage amount and that this refinance doesn't make sense.",
                "I was working with AmeriSave on a refinance, and after reading your report of they are doing the same thing. They intentionally mislead me on every step of the process -low interest rate, closing fees, which changed during the process. I had to pay up front just to move forward with the process, they assured me that this would be a quick, and that I had plenty of equity in my house, and that I had nothing to worry about. I started the process around and went back and forth with them until the loan was about to close on. The last phone call I had with them on the  the lender agreed with me and said they missed looking at my initial mortgage amount and that this refinance doesn't make sense. I spoke with them several times." ,
                "I was working with AmeriSave on a refinance, and after reading your report of they are doing the same thing. They intentionally mislead me on every step of the process -low interest rate, closing fees, which changed during the process. I had to pay up front just to move forward with the process, they assured me that this would be a quick, and that I had plenty of equity in my house,, and that I had nothing to worry about. I started the process around and went back and forth with them until the loan was about to close on. The last phone call I had with them on the  the lender agreed with me and said they missed looking at my initial mortgage amount and that this refinance doesn't make sense. I spoke with them several times. The response was very good.",
                "They intentionally mislead me on every step of the process -low interest rate, closing fees, which changed during the process.",
                "I was working with AmeriSave with on a refinance, and after reading your report of they are doing the same thing. They intentionally mislead me on every step of the closing fees, which changed during the process. I had to pay up front {$500.00} just to move forward with the process, they assured me that this would be a quick, and that I had plenty of equity in my house,, and that I had nothing to worry about. I started the process around XX/XX/XXXX and went back and forth with them until the loan was about to close on XX/XX/XXXX. I spoke with them several times ( every time I was sent new disclosure documents ) because I couldn't see how the numbers were adding up and how it would benefit my current mortgage amount. They assured me that I would be saving more than {$300.00} a month on my new mortgage, plus being able to write off the amount I spent in points on my taxes, skip a few payment ( they really dangled the carrot ), well when I got my final closing documents it was revealed ( only after I crunched more numbers ) less than a savings of {$27.00} dollars a month and would cost me over {$8000.00} in closing fees. The last phone call I had with them on the XX/XX/XXXX the lender agreed with me and said they missed looking at my initial mortgage amount and that YES this refinance doesn't make sense. I'm demanding that they refund my {$500.00}. This is just a brief description of what happened.",
                "I was working with AmeriSave with on a refinance, and after reading your report of they are doing the same thing.",
                "They intentionally mislead me on every step of the closing fees.",
                "On or about BB & T sent a letter to me stating they have suspended my ability to obtain advances on my line of credit. They stated because I have delinquent credit obligations with other creditors. This info was obtained by BB & T thru XXXX. I pulled my credit using the Annual Credit report website and chose XXXX. My credit history with all but one creditor was in good standing. The creditor in question was a card my wife and I had in both our names. She put some dental work on the card and forgot to pay it and didn't even realize she was behind until my line of credit was frozen. I called BB & T XXXX PA Branch and spoke with XXXX XXXX NMLS ID XXXX. She was also the person I spoke with when I applied and obtained the loan in the first place back in XXXX/XXXX  2019. She looked into it and she called me back, said that the line of credit would be frozen for a year. I was in shock. I have plans for this money in the next 12 months and now everything must be placed on hold. She suggested a formal request to unfreeze the a/c to escalate the problem in the BB & T company. I received a notice stating that again my line was frozen because of the same delinquent account and would remain frozen. Also, this loan was opened with a variable rate. I asked if the could be fixed so I can build a payment plan to pay the line of credit off as soon as I can. She looked into it and got back to me, she stated that no, the loan can not be changed from a variable to a fixed for the same reason as the suspension, delinquent account. The delinquent account that caused this mess is now caught up with the payments and in good standing. My credit report reflects this. Its outrageous. I work hard on my credit to be in good standing. All of my other accounts are impeccable. One credit card with a {$600.00} balance that was grossly overlooked and my HELOC and all the plans I have for it is in jeopardy. How I want this to be resolved is 1. I want the suspension to be lifted from my HELOC and be able to use the funds immediately and 2. be able to lock the HELOC into a fixed rate loan.",
                 "I am writing to you about my credit score  of XXXX that was reported to XXXX XXXX MasterCard, by Experian. I feel my credit score should be XXXX, because I dont owe anyone. I also pay off all my bills every month ; I also have excellent credit for the last 30 years. When I call Experian ( XXXX ) I am told that they can not give me that information ; they can only sell me a credit score. I have the right to dispute inaccurate information. Why is my credit score not higher than XXXX? Report # XXXX"]
list_tokenized_test_p = tokenizer.texts_to_sequences(to_predict)
X_te_p = pad_sequences(list_tokenized_test_p, maxlen=maxlen)
a = model2.predict(X_te_p)

In [38]:
type(a)
flat_list = [str(round(item, 2)) for sublist in a for item in sublist]
print(a)

[[0.42269626]
 [0.8136003 ]
 [0.24927393]
 [0.9692651 ]
 [0.8063527 ]
 [0.61273986]
 [0.16116282]
 [0.34541968]
 [0.09018537]
 [0.12067261]
 [0.24269134]
 [0.22306785]
 [0.34178507]
 [0.33900744]
 [0.3332823 ]
 [0.56709284]
 [0.47844955]]


In [39]:

demo = pd.DataFrame(data={'text':to_predict,'Prob':flat_list})
demo.to_csv('demo_results.csv',index=False)

In [40]:
demo

,Prob,text
0,0.42,"Although fairly involving as far as it goes , ..."
1,0.81,What saves this deeply affecting film from bei...
2,0.25,The modern-day royals have nothing on these gu...
3,0.97,"But believe it or not , it 's one of the most ..."
4,0.81,It was very good
5,0.61,It was really good
6,0.16,"It was really bad,"
7,0.35,It is ok to keep them
8,0.09,"I was working with AmeriSave on a refinance, a..."
9,0.12,"I was working with AmeriSave on a refinance, a..."


### Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(X_t,y)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(X_t,y)
print(lr_tfidf)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [42]:
X_t

array([[  42, 1919,   15, ...,   35,   17,  106],
       [   3,  870,  368, ...,   60,  420, 1738],
       [   3,  152,   16, ...,   13, 4441,   21],
       ...,
       [ 162,  508,   22, ...,    6,   40,   79],
       [  57, 1018,    1, ..., 1018, 1041, 1096],
       [  99,  448,   24, ...,   72,  271,    7]], dtype=int32)

In [43]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(X_te)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(X_te)
print(lr_tfidf_predict)

[0 1 0 ... 1 1 0]
[0 1 0 ... 1 1 0]


In [44]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(lr_bow_predict, y_test)
print("lr_bow_score :",lr_bow_score)


lr_bow_score : 0.5195643404598629


In [47]:
ls

 Applying_a_pretrained_Sentiment_model_to_the_data.ipynb
 BiDirectionalLSTM_Sentiment.ipynb
 CFPB_Final.csv
 CFPB_Gradient.ipynb
 clean_full.zip
 complaints-2019-09-13_09_37.7z
 complaints-2019-09-13_09_37.csv
 Consumer_Complaints.csv
 Copy_of_Mortgage_Data_Shared_Notebook.ipynb
 Copy_of_original_for_visualisations.ipynb
 demo_results.csv
'git prod'/
 LDA.ipynb
 LDA-Mortgage.ipynb
 LDA_Vikas/
 model/
 model.h5
'Sam'\''s_copy_(Base_Notebook)_of_Multi_Class_Text_Classification_LSTM_Consumer_complaints.ipynb'
 tokenizer.pickle
 training_2.csv
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled.ipynb
 UST_SmartOps_CFPB.pptx


In [48]:
pwd

'/home/tom/Sprint_6'